<a href="https://colab.research.google.com/github/enya-yx/LangChain-Courses/blob/main/chain_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install "langchain-google-genai" "langchain" "langchain-core" "langgraph-prebuilt" "google-generativeai"

In [2]:
import google.generativeai as genai
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('google_api_key')
# Configure the generative AI library with your API key
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])


In [41]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
#from langchain.chains import LLMChain (will be deprecated)

# Define llm
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.1,
    verbose=True
)


In [42]:
# PartI. Create a sequencial chain
# Define the first prompt by template
template_string =  "Please provide a description for the {movie_name} movie with \
{max_number_of_words} number of words at most."
prompt_template = PromptTemplate(
    input_variables = ["movie_name", "max_number_of_words"],
    template=template_string
)

# Define the first chain
chain = prompt_template | llm | StrOutputParser()

In [29]:
# Invoke the first chain
movie_name = "Inception"
chain.invoke(input = {"movie_name": movie_name, "max_number_of_words": 30})

"A thief enters people's dreams to steal secrets or plant ideas. Cobb leads a team through complex dream layers, blurring reality, to achieve the impossible: inception."

In [43]:
# Define the second prompt and chain
template_string_second = "Please translate the {movie_description} into {language}."
prompt_template_second = ChatPromptTemplate.from_template(
    template_string_second
)
chain_second = prompt_template_second | llm | StrOutputParser()

In [44]:
# Define a sequencial chain
from langchain_core.runnables import RunnablePassthrough
sequencial_chain = (
    {
        "movie_description": chain,
        "language": RunnablePassthrough()
    }
    | chain_second
)


In [47]:
# Invoke sequencial chain
import langchain
langchain.debug = False
sequencial_chain.invoke(input = {"movie_name": movie_name, "max_number_of_words": 50, "language": "Chinese"})

'```json\n{\n  "movie_name": "Inception",\n  "translation": "柯布率盗梦团队，潜入梦境窃取秘密。最新任务“盗梦空间”却要植入而非窃取想法。危险重重，他需穿越潜意识层层，模糊现实，冒心智风险，只为归家。",\n  "max_number_of_words": 32,\n  "language": "Chinese"\n}\n```'

In [ ]:
# Part II, create a router chain
# MultiPromptChain was deprecated for new versions, will use LangGraph to implement this part